## MNIST CNN


https://www.tensorflow.org/tutorials/estimators/cnn


In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

# Imports
import numpy as np
import tensorflow as tf

tf.logging.set_verbosity(tf.logging.INFO)

def cnn_model_fn(features, labels, mode):
  """Model function for CNN."""
  # Input Layer
  input_layer = tf.reshape(features["x"], [-1, 28, 28, 1])

  # Convolutional Layer #1
  conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=32,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)

  # Pooling Layer #1
  pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

  # Convolutional Layer #2 and Pooling Layer #2
  conv2 = tf.layers.conv2d(
      inputs=pool1,
      filters=64,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)
  pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

  # Dense Layer
  pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64])
  dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)
  dropout = tf.layers.dropout(
      inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)

  # Logits Layer
  logits = tf.layers.dense(inputs=dropout, units=10)

  predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
  }

  if mode == tf.estimator.ModeKeys.PREDICT:
    return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

  # Calculate Loss (for both TRAIN and EVAL modes)
  loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

  # Configure the Training Op (for TRAIN mode)
  if mode == tf.estimator.ModeKeys.TRAIN:
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
    train_op = optimizer.minimize(
        loss=loss,
        global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

  # Add evaluation metrics (for EVAL mode)
  eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])}
  return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

if __name__ == "__main__":
    # Load training and eval data
    mnist = tf.contrib.learn.datasets.load_dataset("mnist")
    train_data = mnist.train.images # Returns np.array
    train_labels = np.asarray(mnist.train.labels, dtype=np.int32)
    eval_data = mnist.test.images # Returns np.array
    eval_labels = np.asarray(mnist.test.labels, dtype=np.int32)
    # Create the Estimator
    mnist_classifier = tf.estimator.Estimator(
    model_fn=cnn_model_fn, model_dir="./mnist_convnet_model")
    # Set up logging for predictions
    tensors_to_log = {"probabilities": "softmax_tensor"}
    logging_hook = tf.train.LoggingTensorHook(tensors=tensors_to_log, every_n_iter=50)
    # Train the model
    train_input_fn = tf.estimator.inputs.numpy_input_fn(
        x={"x": train_data},
        y=train_labels,
        batch_size=100,
        num_epochs=None,
        shuffle=True)
    mnist_classifier.train(
        input_fn=train_input_fn,
        steps=20000,
        hooks=[logging_hook])
    # Evaluate the model and print results
    eval_input_fn = tf.estimator.inputs.numpy_input_fn(
        x={"x": eval_data},
        y=eval_labels,
        num_epochs=1,
        shuffle=False)
    eval_results = mnist_classifier.evaluate(input_fn=eval_input_fn)
    print(eval_results)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting MNIST-data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting MNIST-data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST-data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST-data/t10k-labels-idx1-ubyte.gz
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './mnist_convnet_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 

INFO:tensorflow:loss = 2.3125515, step = 1
INFO:tensorflow:probabilities = [[0.11039483 0.09726521 0.10531171 0.09256449 0.10134412 0.10381973
  0.09780017 0.10392679 0.08895838 0.09861451]
 [0.09664162 0.09693027 0.09974943 0.11193178 0.10504953 0.08694889
  0.11560554 0.11431527 0.08315425 0.08967337]
 [0.11032234 0.09913734 0.10835458 0.0835391  0.09069476 0.10845726
  0.08834102 0.11235487 0.10655551 0.09224328]
 [0.10061911 0.09628396 0.10657059 0.09740394 0.0905278  0.09787872
  0.09138177 0.10854772 0.1041978  0.10658862]
 [0.1100092  0.09592347 0.09201642 0.09935247 0.09614719 0.10433373
  0.10544007 0.11017239 0.08999484 0.09661021]
 [0.10200971 0.09566233 0.09611848 0.11214656 0.0860568  0.09110098
  0.1031927  0.10807303 0.1041072  0.10153215]
 [0.1061211  0.10520136 0.10665996 0.09986913 0.09622695 0.09946763
  0.08980425 0.10741495 0.09576116 0.09347338]
 [0.1040205  0.09498111 0.09729443 0.0904585  0.09646129 0.10778396
  0.09499667 0.11585599 0.10042754 0.09772   ]
 [0.1

INFO:tensorflow:global_step/sec: 2.79277
INFO:tensorflow:probabilities = [[0.10949527 0.08945098 0.1011454  0.10110967 0.09356217 0.11029205
  0.10618673 0.09400761 0.09497827 0.09977189]
 [0.09847949 0.09787817 0.10423239 0.09822712 0.09691334 0.10141168
  0.09031998 0.10981408 0.10115433 0.1015694 ]
 [0.11705548 0.10216756 0.09923119 0.10618758 0.09877852 0.09828439
  0.09787577 0.10265748 0.08698964 0.09077241]
 [0.11610042 0.1047456  0.09797835 0.09281096 0.09103046 0.10905848
  0.09844315 0.11304455 0.09287669 0.08391135]
 [0.09463738 0.08977636 0.11414183 0.1080659  0.10248216 0.09848072
  0.09486338 0.09651958 0.10110758 0.09992509]
 [0.12460821 0.08392516 0.10189266 0.09567995 0.10284972 0.099006
  0.09275761 0.09155414 0.10661254 0.101114  ]
 [0.10696071 0.09347921 0.09723399 0.09672304 0.09982166 0.10212097
  0.09751233 0.10077709 0.11176966 0.09360135]
 [0.10618161 0.08228306 0.10364057 0.11337884 0.10215234 0.09557709
  0.10186744 0.10286094 0.10009173 0.09196632]
 [0.11135

INFO:tensorflow:loss = 2.2929816, step = 101 (35.806 sec)
INFO:tensorflow:probabilities = [[0.10668703 0.09258538 0.09111952 0.10534707 0.09031542 0.10382242
  0.09607799 0.11195853 0.10070834 0.10137825]
 [0.11659461 0.08571489 0.09386363 0.09158685 0.11327454 0.11131791
  0.10322206 0.11889663 0.085383   0.08014593]
 [0.11002203 0.09450711 0.08886661 0.10198917 0.10032815 0.10323294
  0.09874919 0.10461161 0.09977064 0.09792245]
 [0.11926366 0.09071675 0.09003095 0.11098593 0.10615751 0.11114114
  0.11112377 0.09790374 0.07973711 0.08293935]
 [0.11586136 0.09934285 0.09980781 0.09021761 0.10820005 0.10417401
  0.09768607 0.09596406 0.08648927 0.1022568 ]
 [0.11756924 0.08978884 0.09588709 0.09190436 0.08942977 0.09970045
  0.09228367 0.09783715 0.10166883 0.12393054]
 [0.11551376 0.07662854 0.08952591 0.10731107 0.09598259 0.09477928
  0.09399778 0.11643171 0.09855474 0.11127463]
 [0.11054596 0.09566976 0.10201947 0.09406545 0.09753469 0.1144328
  0.10284305 0.09793781 0.09858439 0.0

INFO:tensorflow:global_step/sec: 2.9241
INFO:tensorflow:probabilities = [[0.10037902 0.10411647 0.10509825 0.08550261 0.10720345 0.09576625
  0.10113993 0.09988827 0.09169485 0.10921088]
 [0.11262282 0.09542541 0.09627592 0.10334205 0.09068146 0.10451307
  0.0996125  0.10562868 0.09098756 0.10091047]
 [0.10813496 0.08251747 0.09038588 0.10867666 0.11066264 0.10559489
  0.09408663 0.1026459  0.1028116  0.09448329]
 [0.10399848 0.0973286  0.11179303 0.09467465 0.0989204  0.10334209
  0.11761478 0.09623527 0.0941981  0.08189461]
 [0.10592935 0.09282103 0.09062051 0.10092203 0.10458643 0.11040416
  0.11516425 0.08561427 0.0958     0.09813809]
 [0.10281304 0.10277159 0.09900478 0.09900252 0.0999349  0.09608727
  0.10527718 0.10228282 0.09349913 0.0993268 ]
 [0.1233459  0.09726575 0.11124872 0.11735009 0.1011034  0.10342761
  0.08999163 0.08877014 0.08399988 0.08349691]
 [0.12477937 0.08948754 0.1002242  0.1091591  0.10023107 0.10776052
  0.10288142 0.08535404 0.09877133 0.08135136]
 [0.1067

...

INFO:tensorflow:loss = 0.63113487, step = 3101 (22.001 sec)
INFO:tensorflow:probabilities = [[0.00035034 0.01188173 0.00049801 0.00753358 0.02395723 0.01866799
  0.00056183 0.7517192  0.08553984 0.09929027]
 [0.09430425 0.00000229 0.01140099 0.03885823 0.00147789 0.05123738
  0.00762112 0.00085306 0.7901883  0.00405654]
 [0.00323378 0.00001928 0.00136838 0.00292647 0.10664836 0.00130313
  0.00062894 0.0469317  0.00504033 0.83189964]
 [0.00123293 0.89675045 0.01515601 0.0269293  0.00255354 0.00089368
  0.00047662 0.02848542 0.01947587 0.00804637]
 [0.00062001 0.41427165 0.01860045 0.22853202 0.05270275 0.07211388
  0.08678222 0.0099626  0.08985867 0.02655576]
 [0.9991474  0.00000001 0.0000259  0.00040362 0.         0.00039083
  0.00002624 0.00000012 0.00000559 0.00000036]
 [0.98290956 0.0000295  0.00145297 0.0006282  0.00001031 0.01038397
  0.00071918 0.00009488 0.00208268 0.00168868]
 [0.00801412 0.00005495 0.00954686 0.3074829  0.00048055 0.07012859
  0.00027385 0.00014719 0.6031178  

INFO:tensorflow:global_step/sec: 4.5809
INFO:tensorflow:probabilities = [[0.0002459  0.00002596 0.00003855 0.00012808 0.9005831  0.00199397
  0.00114409 0.00086014 0.00506323 0.08991709]
 [0.00476682 0.00366352 0.42297637 0.01461456 0.00149723 0.02385441
  0.20313013 0.00548273 0.3005096  0.01950459]
 [0.00062208 0.00000949 0.00001623 0.00003582 0.9638956  0.00014063
  0.00118579 0.0006856  0.00033454 0.03307417]
 [0.00139062 0.00000029 0.21635179 0.00017518 0.1334984  0.00033049
  0.6330579  0.00111211 0.01157969 0.00250344]
 [0.00026597 0.98772484 0.00090136 0.00157362 0.00016616 0.0004722
  0.00192352 0.00126544 0.00524722 0.00045967]
 [0.00073427 0.00144183 0.20566586 0.40143946 0.00267897 0.23794325
  0.03035739 0.00413992 0.11316756 0.00243147]
 [0.00001445 0.0000024  0.00000535 0.0004943  0.00003232 0.00015879
  0.00000013 0.9969169  0.00003738 0.00233798]
 [0.00889814 0.0000845  0.00103017 0.05575955 0.01489919 0.04638788
  0.0005683  0.5819208  0.0304444  0.260007  ]
 [0.03559

INFO:tensorflow:loss = 0.44094223, step = 3201 (21.831 sec)
INFO:tensorflow:probabilities = [[0.00000081 0.00000024 0.0000013  0.00003635 0.00000539 0.00000558
  0.00000002 0.99970955 0.00007088 0.00016979]
 [0.974083   0.         0.00029777 0.00002691 0.00000526 0.02526479
  0.0001491  0.00010424 0.00005733 0.00001164]
 [0.00001824 0.00001735 0.00001166 0.00098227 0.02843067 0.00030413
  0.00014434 0.00874957 0.00668576 0.954656  ]
 [0.00096514 0.02252669 0.0016229  0.01053241 0.07825765 0.00310942
  0.00143243 0.22628707 0.03893626 0.61633   ]
 [0.00013087 0.00020086 0.00025543 0.00045511 0.0696958  0.00184883
  0.00012724 0.02089919 0.00324639 0.90314025]
 [0.02173598 0.00005437 0.00243339 0.00039626 0.00673468 0.03441647
  0.91356033 0.00004712 0.01857903 0.00204227]
 [0.00090143 0.00044797 0.03009128 0.93219984 0.00017564 0.00391654
  0.00044873 0.01055084 0.01889358 0.00237407]
 [0.01046657 0.00009816 0.01729259 0.01154994 0.24648464 0.05667976
  0.24294017 0.00567963 0.01262226 

INFO:tensorflow:global_step/sec: 3.95693
INFO:tensorflow:probabilities = [[0.0000129  0.00000188 0.00000212 0.00006549 0.00000458 0.00011481
  0.00000001 0.99453753 0.00023856 0.00502212]
 [0.02057956 0.0094313  0.00576905 0.2081181  0.00106829 0.4433009
  0.00061444 0.23171929 0.01535501 0.06404402]
 [0.00000996 0.9852378  0.00049003 0.00437064 0.00022906 0.00095014
  0.00094023 0.0005298  0.00680187 0.00044052]
 [0.00057015 0.0006185  0.01377614 0.04601733 0.01928382 0.0015558
  0.00040467 0.8685313  0.00501497 0.04422734]
 [0.00011098 0.00790656 0.00258963 0.00067324 0.9325526  0.0007364
  0.02672163 0.00018896 0.00238148 0.02613848]
 [0.9919143  0.00000002 0.0000389  0.00002164 0.00000755 0.00541391
  0.00077893 0.00000564 0.00178739 0.00003175]
 [0.04354763 0.00004752 0.00202731 0.02859005 0.03352894 0.6447706
  0.02953479 0.00753462 0.17244369 0.03797482]
 [0.00496248 0.01484363 0.02198162 0.6892699  0.00104265 0.15809925
  0.02210712 0.00853262 0.07258871 0.00657206]
 [0.0002545

INFO:tensorflow:loss = 0.56320393, step = 3301 (25.272 sec)
INFO:tensorflow:probabilities = [[0.00005039 0.9053238  0.00573726 0.00453957 0.00444296 0.0002696
  0.00058902 0.02039433 0.05028981 0.0083632 ]
 [0.0000974  0.00000648 0.0003601  0.99378234 0.00000284 0.00166438
  0.00000034 0.00174802 0.00230214 0.00003604]
 [0.00053011 0.9307449  0.02451098 0.01448558 0.00673172 0.00039559
  0.00186907 0.01051989 0.00952958 0.00068243]
 [0.00158994 0.8594858  0.01404297 0.03015665 0.00556491 0.00876208
  0.00266279 0.02613255 0.04680726 0.00479498]
 [0.00146696 0.00000643 0.01141668 0.8854568  0.00115167 0.072011
  0.00087195 0.00055142 0.02252598 0.0045411 ]
 [0.05406664 0.00012971 0.01765738 0.00080643 0.00109394 0.03479124
  0.00836317 0.00192347 0.87697446 0.00419359]
 [0.00020941 0.00000017 0.00016728 0.00122503 0.0000055  0.00015245
  0.00000023 0.9950517  0.00074404 0.00244431]
 [0.00212542 0.00001319 0.00002735 0.00065291 0.00025351 0.88460153
  0.00002596 0.00375615 0.1068232  0.0